# EDA

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import datetime

Data can be accessed from this google drive link. The "play log" contains the records of users active session. There are 9 mini games inside the app "Lengbear", an social casino games made and publish exclusively for Cambodia. The transactions log is the records of user in app purchase.

In [43]:
# Columns name for player log, transactions and match length by GameID provided by data owner

columns_play_log=('Sequence','UserID', 'GameID', 'Level', 'WinNo', 'DrawNo', 'LostNo', 'WinAmt', 'LostAmt', 'Date', 'Currency_Type1', 'Currency_Type2')
columns_transactions=('UserID','Amount','Chips','Date', 'Channel')

In [44]:
#making the dataframe for GameID and Match_Length (data provided by game owner)

#define BINH                9001: 80 seconds
#define POKDENG             9002: 80 seconds
#define BAUCUA              9005: 20 seconds
#define SHOW                9004: 80 seconds
#define XOCDIA              9003: 20 seconds
#define SIKU                9006: 80 seconds
#define SLOT_20_LINE        9008: 20 seconds
#define SLOT_100_LINE       1008: 20 seconds
#define TIENLEN_CAM         9009: 120 seconds

Game_Code_Length=pd.DataFrame({'GameID':[9001, 9002, 9005, 9004, 9003, 9006, 9008, 1008, 9009],
                               'Match_Length':[75,75,20,75,20,75,20,20,120]})
Game_Code_Length

,GameID,Match_Length
0,9001,75
1,9002,75
2,9005,20
3,9004,75
4,9003,20
5,9006,75
6,9008,20
7,1008,20
8,9009,120


In [45]:
# Load the play log file and check if there is any missing values

play_log_df=pd.read_csv('data/PlayerLog.csv', names=columns_play_log)
play_log_df.isna().sum()

Sequence          0
UserID            0
GameID            0
Level             0
WinNo             0
DrawNo            0
LostNo            0
WinAmt            0
LostAmt           0
Date              0
Currency_Type1    0
Currency_Type2    0
dtype: int64

In [46]:
# Load the transactions log data and check if there is any missing values

transactions_df=pd.read_csv('data/TransactionLog.csv', names=columns_transactions)
transactions_df.isna().sum()

UserID     0
Amount     0
Chips      0
Date       0
Channel    0
dtype: int64

In [47]:
# Remove redundant columns and set the time to regular format

play_log_df.drop(['Sequence','Currency_Type1','Currency_Type2'], axis=1, inplace=True)
play_log_df['Date']=pd.to_datetime(play_log_df['Date'].str[:10])
play_log_df.head()

,UserID,GameID,Level,WinNo,DrawNo,LostNo,WinAmt,LostAmt,Date
0,5894561,9009,1000,0,0,1,0,24964,2020-05-09
1,6047531,9009,20,2,0,0,515,0,2020-05-09
2,4972744,9009,100,1,0,3,1748,7800,2020-05-09
3,5608136,9009,1000,9,0,15,128680,137000,2020-05-09
4,4725768,9009,100,10,0,34,18272,27100,2020-05-09


In [48]:
#getting the Match_Length column ready to calculate the active session of user.

play_log_df=play_log_df.merge(Game_Code_Length, on='GameID', how='left')
play_log_df.head()

,UserID,GameID,Level,WinNo,DrawNo,LostNo,WinAmt,LostAmt,Date,Match_Length
0,5894561,9009,1000,0,0,1,0,24964,2020-05-09,120
1,6047531,9009,20,2,0,0,515,0,2020-05-09,120
2,4972744,9009,100,1,0,3,1748,7800,2020-05-09,120
3,5608136,9009,1000,9,0,15,128680,137000,2020-05-09,120
4,4725768,9009,100,10,0,34,18272,27100,2020-05-09,120


In [49]:
#counting the number of games play in each log and get the actual active play time of users.

play_log_df['Games_Played']=play_log_df['WinNo']+play_log_df['DrawNo']+play_log_df['LostNo']
play_log_df['Active_Time']=play_log_df['Games_Played']*play_log_df['Match_Length']
play_log_df

,UserID,GameID,Level,WinNo,DrawNo,LostNo,WinAmt,LostAmt,Date,Match_Length,Games_Played,Active_Time
0,5894561,9009,1000,0,0,1,0,24964,2020-05-09,120,1,120
1,6047531,9009,20,2,0,0,515,0,2020-05-09,120,2,240
2,4972744,9009,100,1,0,3,1748,7800,2020-05-09,120,4,480
3,5608136,9009,1000,9,0,15,128680,137000,2020-05-09,120,24,2880
4,4725768,9009,100,10,0,34,18272,27100,2020-05-09,120,44,5280
...,...,...,...,...,...,...,...,...,...,...,...,...
1768635,4969001,9009,20,2,0,0,1435,0,2020-05-10,120,2,240
1768636,812253,9009,5000,0,0,3,0,335000,2020-05-10,120,3,360
1768637,6037451,9009,20,0,0,1,0,294,2020-05-10,120,1,120
1768638,4449836,9009,5000,0,0,1,0,135000,2020-05-10,120,1,120


In [50]:
# Getting t0 and t4 timestamp

t0=play_log_df['Date'].min()
t4=t0+datetime.timedelta(days=3)
tchurn=t0+datetime.timedelta(days=4)
print(f't0={t0} \nt4={t4} \ntchurn={tchurn}')

t0=2020-05-01 00:00:00 
t4=2020-05-04 00:00:00 
tchurn=2020-05-05 00:00:00


In [51]:
# Getting users who played on t0

play_log_t0=play_log_df.loc[play_log_df['Date']==t0]
t0_users=pd.DataFrame(play_log_t0['UserID'].unique())
t0_users.columns=['UserID']
t0_users

,UserID
0,4775802
1,5929933
2,5728479
3,5725542
4,5942114
...,...
56158,5155914
56159,5930800
56160,4858147
56161,5345776


In [52]:
# Slice out the dataframe that contains user playlog from t0 -> t4.

play_log_t0_t4=play_log_df.loc[(play_log_df['Date']>=t0) & (play_log_df['Date']<=t4)]

In [53]:
# Compute the Winning rate and Drawing rate, Losing Rate, Winning Amount, Losing Amount

WinNo=pd.DataFrame(play_log_t0_t4.groupby(['UserID']).sum()['WinNo'])
DrawNo=pd.DataFrame(play_log_t0_t4.groupby(['UserID']).sum()['DrawNo'])
LostNo=pd.DataFrame(play_log_t0_t4.groupby(['UserID']).sum()['LostNo'])
WinAmt=pd.DataFrame(play_log_t0_t4.groupby(['UserID']).sum()['WinAmt'])
LostAmt=pd.DataFrame(play_log_t0_t4.groupby(['UserID']).sum()['LostAmt'])
Games_Played=pd.DataFrame(play_log_t0_t4.groupby(['UserID']).sum()['Games_Played'])
Active_Time=pd.DataFrame(play_log_t0_t4.groupby(['UserID']).sum()['Active_Time'])

In [54]:
play_log_summary=WinNo.merge(DrawNo, on='UserID', how='left')\
                        .merge(LostNo, on='UserID', how='left')\
                        .merge(WinAmt, on='UserID', how='left')\
                        .merge(LostAmt, on='UserID', how='left')\
                        .merge(Games_Played, on='UserID', how='left')\
                        .merge(Active_Time, on='UserID', how='left')
play_log_summary.columns=['WinNo', 'DrawNo', 'LostNo', 'WinAmt', 'LostAmt', 'Games_Played', 'Active_Time']
play_log_summary.shape

(118810, 7)

In [55]:
play_log_summary=play_log_summary.reset_index(level='UserID')

In [56]:
play_log_summary

,UserID,WinNo,DrawNo,LostNo,WinAmt,LostAmt,Games_Played,Active_Time
0,1980,0,0,3,0,417197,3,60
1,3452,7,0,12,6800050,8971800,19,1095
2,3700,73,0,72,18404347,21637393,145,10765
3,4079,10,0,13,10639687,17156250,23,1725
4,4732,3,0,12,10726,52726,15,1125
...,...,...,...,...,...,...,...,...
118805,5986120,4,0,22,8400,18000,26,520
118806,5986122,2,0,1,476,876,3,225
118807,5986131,3,0,6,7500,4590,9,180
118808,5986132,1,0,2,100,130,3,60


In [57]:
# Getting the right format for date in transaction data

transactions_df['Date']=pd.to_datetime(transactions_df['Date'].str[:10])

In [58]:
# Slice out the time for study

transactions_df_t0_t4=transactions_df.loc[(transactions_df['Date']>=t0) & (transactions_df['Date']<=t4)]
transactions_df_t0_t4.shape

(17921, 5)

In [59]:
# Sum in-game purchase by users during t0 -> t4

transactions_df_t0_t4=pd.DataFrame(transactions_df_t0_t4.groupby(['UserID']).sum()['Amount'])
transactions_df_t0_t4.reset_index(level='UserID')

,UserID,Amount
0,1980,0.2
1,3700,2.0
2,12775,0.5
3,23957,0.5
4,42563,26.0
...,...,...
8239,5985700,0.2
8240,5985746,0.2
8241,5985766,0.2
8242,5985830,0.2


In [60]:
play_log_summary=play_log_summary.merge(transactions_df_t0_t4, on='UserID', how='left')
play_log_summary=play_log_summary.fillna(0)

In [61]:
play_log_summary['WinningRate']=play_log_summary['WinNo']/play_log_summary['Games_Played']
play_log_summary['DrawRate']=play_log_summary['DrawNo']/play_log_summary['Games_Played']
play_log_summary['LosingRate']=play_log_summary['LostNo']/play_log_summary['Games_Played']
play_log_summary

,UserID,WinNo,DrawNo,LostNo,WinAmt,LostAmt,Games_Played,Active_Time,Amount,WinningRate,DrawRate,LosingRate
0,1980,0,0,3,0,417197,3,60,0.2,0.000000,0.0,1.000000
1,3452,7,0,12,6800050,8971800,19,1095,0.0,0.368421,0.0,0.631579
2,3700,73,0,72,18404347,21637393,145,10765,2.0,0.503448,0.0,0.496552
3,4079,10,0,13,10639687,17156250,23,1725,0.0,0.434783,0.0,0.565217
4,4732,3,0,12,10726,52726,15,1125,0.0,0.200000,0.0,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...
118805,5986120,4,0,22,8400,18000,26,520,0.0,0.153846,0.0,0.846154
118806,5986122,2,0,1,476,876,3,225,0.0,0.666667,0.0,0.333333
118807,5986131,3,0,6,7500,4590,9,180,0.0,0.333333,0.0,0.666667
118808,5986132,1,0,2,100,130,3,60,0.0,0.333333,0.0,0.666667


In [62]:
play_log_tchurn=play_log_df.loc[play_log_df['Date']==tchurn]

In [63]:
# Getting target 

final_play_log=play_log_summary.assign(Churn=play_log_summary.UserID.isin(play_log_tchurn.UserID).astype(int))
final_play_log

,UserID,WinNo,DrawNo,LostNo,WinAmt,LostAmt,Games_Played,Active_Time,Amount,WinningRate,DrawRate,LosingRate,Churn
0,1980,0,0,3,0,417197,3,60,0.2,0.000000,0.0,1.000000,0
1,3452,7,0,12,6800050,8971800,19,1095,0.0,0.368421,0.0,0.631579,1
2,3700,73,0,72,18404347,21637393,145,10765,2.0,0.503448,0.0,0.496552,1
3,4079,10,0,13,10639687,17156250,23,1725,0.0,0.434783,0.0,0.565217,0
4,4732,3,0,12,10726,52726,15,1125,0.0,0.200000,0.0,0.800000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118805,5986120,4,0,22,8400,18000,26,520,0.0,0.153846,0.0,0.846154,0
118806,5986122,2,0,1,476,876,3,225,0.0,0.666667,0.0,0.333333,1
118807,5986131,3,0,6,7500,4590,9,180,0.0,0.333333,0.0,0.666667,1
118808,5986132,1,0,2,100,130,3,60,0.0,0.333333,0.0,0.666667,1


In [64]:

X=final_play_log.iloc[:,:-1]
X=X.drop(['UserID'], axis=1)
X

,WinNo,DrawNo,LostNo,WinAmt,LostAmt,Games_Played,Active_Time,Amount,WinningRate,DrawRate,LosingRate
0,0,0,3,0,417197,3,60,0.2,0.000000,0.0,1.000000
1,7,0,12,6800050,8971800,19,1095,0.0,0.368421,0.0,0.631579
2,73,0,72,18404347,21637393,145,10765,2.0,0.503448,0.0,0.496552
3,10,0,13,10639687,17156250,23,1725,0.0,0.434783,0.0,0.565217
4,3,0,12,10726,52726,15,1125,0.0,0.200000,0.0,0.800000
...,...,...,...,...,...,...,...,...,...,...,...
118805,4,0,22,8400,18000,26,520,0.0,0.153846,0.0,0.846154
118806,2,0,1,476,876,3,225,0.0,0.666667,0.0,0.333333
118807,3,0,6,7500,4590,9,180,0.0,0.333333,0.0,0.666667
118808,1,0,2,100,130,3,60,0.0,0.333333,0.0,0.666667


In [65]:
X=X.fillna(0)

In [66]:
y=final_play_log.iloc[:,-1]

In [67]:
y.value_counts()

0    80464
1    38346
Name: Churn, dtype: int64

In [74]:
# Create a train/test split with 80% train, 20% test 

X_remainder, X_test, y_remainder, y_test=train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [75]:
from sklearn.linear_model import LogisticRegression

# 1. Instantiate model
churn_logistic_regression=LogisticRegression(random_state=42, max_iter=10000)

# 2. Fit model
churn_logistic_regression.fit(X_remainder, y_remainder)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [77]:
# 3. Score model
print(f'Score on train: {churn_logistic_regression.score(X_remainder, y_remainder)}')
print(f'Score on test: {churn_logistic_regression.score(X_test, y_test)}')

Score on train: 0.3230473024156216
Score on test: 0.3229105294167158


In [78]:
from sklearn.tree import DecisionTreeClassifier

DT=DecisionTreeClassifier(random_state=42).fit(X_remainder, y_remainder)

# Accuracy scores
print(DT.score(X_remainder, y_remainder))
print(DT.score(X_test, y_test))

0.9883848160929215
0.6930813904553489
